# ✨ MLOps Zoom Camp W2

In [19]:
# importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


In [20]:
import pickle

In [21]:
import mlflow
# mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_tracking_uri("sqlite:///mlflow.db") # needed to run with sqlite
mlflow.set_experiment("NYC-taxi-expirement")

<Experiment: artifact_location='file:///C:/Users/user/Desktop/MLOPS/MLOPS/mlruns/10', creation_time=1685814654256, experiment_id='10', last_update_time=1685814654256, lifecycle_stage='active', name='NYC-taxi-expirement', tags={}>

In [22]:

# reading Parquet file and getting the duration
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)    
    return df

In [23]:
df = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [24]:
# getting duration column
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.std()

46.44530513776499

In [25]:
 df_new = df[(df.duration >= 1) & (df.duration <= 60)]

In [26]:
fraction_records = df_new.shape[0] / df.shape[0]

In [27]:
fraction_records

0.9827547930522406

In [28]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) # one hot encoding
target = 'duration'
y_train = df[target].values
X_train.shape

C:\Users\user\AppData\Local\Temp\ipykernel_15836\1695165724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


(2421440, 515)

In [29]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190685619784

In [30]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [31]:
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts) # one hot encoding
target = 'duration'
y_val = df_val[target].values
X_val.shape

(2918187, 515)

In [32]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786409438115243

In [33]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Reham")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2022-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [35]:
import xgboost as xgb

In [36]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [37]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [38]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [39]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[21:17:32] WARNING: c:\users\dev-admin\croot2\xgboost-split_1675461376218\work\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.63809                                                                                           
[1]	validation-rmse:9.36835                                                                                            
[2]	validation-rmse:8.36074                                                                                            
[3]	validation-rmse:7.90653                                                                                            
[4]	validation-rmse:7.47053                                                                                            
[5]	validation-rmse:7.33518                                                                                            
[6]	validation-rmse:7.18454                                                                              

[66]	validation-rmse:5.66529                                                                                           
[67]	validation-rmse:5.66007                                                                                           
[68]	validation-rmse:5.64259                                                                                           
[69]	validation-rmse:5.63666                                                                                           
[70]	validation-rmse:5.63633                                                                                           
[71]	validation-rmse:5.63473                                                                                           
[72]	validation-rmse:5.63357                                                                                           
[73]	validation-rmse:5.63313                                                                                           
[74]	validation-rmse:5.62763            

[134]	validation-rmse:5.48895                                                                                          
[135]	validation-rmse:5.48862                                                                                          
[136]	validation-rmse:5.48756                                                                                          
[137]	validation-rmse:5.48676                                                                                          
[138]	validation-rmse:5.48666                                                                                          
[139]	validation-rmse:5.48657                                                                                          
[140]	validation-rmse:5.48339                                                                                          
[141]	validation-rmse:5.48088                                                                                          
[142]	validation-rmse:5.47986           

[202]	validation-rmse:5.46261                                                                                          
[203]	validation-rmse:5.46258                                                                                          
[204]	validation-rmse:5.46257                                                                                          
[205]	validation-rmse:5.46250                                                                                          
[206]	validation-rmse:5.46253                                                                                          
[207]	validation-rmse:5.46252                                                                                          
[208]	validation-rmse:5.46247                                                                                          
[209]	validation-rmse:5.46245                                                                                          
[210]	validation-rmse:5.46242           

[270]	validation-rmse:5.46133                                                                                          
[271]	validation-rmse:5.46136                                                                                          
[272]	validation-rmse:5.46134                                                                                          
[273]	validation-rmse:5.46135                                                                                          
[274]	validation-rmse:5.46131                                                                                          
[275]	validation-rmse:5.46133                                                                                          
[276]	validation-rmse:5.46126                                                                                          
[277]	validation-rmse:5.46130                                                                                          
[278]	validation-rmse:5.46125           

...

[102]	validation-rmse:5.65925                                                                                          
[103]	validation-rmse:5.64719                                                                                          
[104]	validation-rmse:5.64647                                                                                          
[105]	validation-rmse:5.64543                                                                                          
[106]	validation-rmse:5.64475                                                                                          
[107]	validation-rmse:5.63281                                                                                          
[108]	validation-rmse:5.63059                                                                                          
[109]	validation-rmse:5.63028                                                                                          
[110]	validation-rmse:5.62969           

[170]	validation-rmse:5.50758                                                                                          
[171]	validation-rmse:5.50741                                                                                          
[172]	validation-rmse:5.50703                                                                                          
[173]	validation-rmse:5.50659                                                                                          
[174]	validation-rmse:5.50394                                                                                          
[175]	validation-rmse:5.49983                                                                                          
[176]	validation-rmse:5.49956                                                                                          
[177]	validation-rmse:5.49949                                                                                          
[178]	validation-rmse:5.49942           

[238]	validation-rmse:5.45622                                                                                          
[239]	validation-rmse:5.45412                                                                                          
[240]	validation-rmse:5.45272                                                                                          
[241]	validation-rmse:5.45267                                                                                          
[242]	validation-rmse:5.45068                                                                                          
[243]	validation-rmse:5.45068                                                                                          
[244]	validation-rmse:5.45059                                                                                          
[245]	validation-rmse:5.44962                                                                                          
[246]	validation-rmse:5.44914           

[306]	validation-rmse:5.43347                                                                                          
[307]	validation-rmse:5.43347                                                                                          
[308]	validation-rmse:5.43298                                                                                          
[309]	validation-rmse:5.43298                                                                                          
[310]	validation-rmse:5.43300                                                                                          
[311]	validation-rmse:5.43284                                                                                          
[312]	validation-rmse:5.43284                                                                                          
[313]	validation-rmse:5.43229                                                                                          
[314]	validation-rmse:5.43228           

KeyboardInterrupt: 

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        